In [1]:
import numpy as np
import pandas as pd

In [1]:
# df = pd.read_pickle('df_all_3.pkl')
# df.head()

In [2]:
# pp_data_train = pd.read_csv('pp_data_train.csv')
# pp_data_train.head()

In [3]:
# y_all = pp_data_train[['row_id', 'incident_id', 'vehicle_id', 'occupant_id', 'injury']]
# y_all.head()

In [4]:
# mapping_y =  {'Unharmed':0, 'Possible':1, 'Minor':2, 'Major':3, 'Fatal':4}

# y_all['injury'] = y_all['injury'].map(mapping_y)
# y_all.head()

In [6]:
X_df_train = df.loc[df['mod_lb'] == 'train', :]
X_df_test = df.loc[df['mod_lb'] == 'test', :]

In [7]:
X_df_train.drop(['mod_lb'], axis = 1, inplace = True)
X_df_test.drop(['mod_lb'], axis = 1, inplace = True)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
print(X_df_train.shape)
print(X_df_test.shape)

(145063, 64)
(289671, 64)


In [9]:
# random sampling based on the 'incident_id'

index = np.unique(X_df_train['incident_id'])
len(index)

64149

In [10]:
index

array([     2,      4,      5, ..., 128186, 128187, 128188], dtype=int64)

In [11]:
from sklearn.model_selection import train_test_split

index_train, index_valid = train_test_split(index, test_size = 0.2, random_state = 42)
print(len(index_train))
print(len(index_valid))

51319
12830


In [5]:
# split X_df_train into X_train and X_valid

# X_df_train.set_index(['incident_id'], inplace = True)
# X_df_train.head()

In [13]:
X_train = X_df_train.loc[index_train, :]
X_valid = X_df_train.loc[index_valid, :]

In [6]:
# X_train.reset_index()
# X_valid.reset_index()

In [17]:
# split y_all into y_train & y_valid

In [1]:
# y_all.set_index(['row_id'], inplace = True)
# y_all.head()

In [16]:
y_train = y_all.loc[X_train['row_id'],: ]
y_valid = y_all.loc[X_valid['row_id'], :]

In [2]:
# y_train.head()

In [3]:
# y_valid.tail()

In [7]:
# y_train.reset_index()
# y_valid.reset_index()

In [20]:
X_train.to_pickle('X_train.pkl')
X_valid.to_pickle('X_valid.pkl')
y_train.to_pickle('y_train.pkl')
y_valid.to_pickle('y_.valid.pkl')

In [32]:
y_train.reset_index(inplace = True)

In [33]:
y_valid.reset_index(inplace = True)

In [34]:
y_train.columns

Index(['row_id', 'incident_id', 'vehicle_id', 'occupant_id', 'injury'], dtype='object')

In [36]:
y_valid.columns

Index(['row_id', 'incident_id', 'vehicle_id', 'occupant_id', 'injury'], dtype='object')

In [37]:
y_train.drop(['row_id', 'incident_id', 'vehicle_id', 'occupant_id'], axis = 1,inplace = True)
y_valid.drop(['row_id', 'incident_id', 'vehicle_id', 'occupant_id'], axis = 1, inplace = True)

In [38]:
print(y_train.columns)
print(y_valid.columns)

Index(['injury'], dtype='object')
Index(['injury'], dtype='object')


In [8]:
# X_train.columns.values

In [59]:
X_train.reset_index(inplace = True)
X_valid.reset_index(inplace = True)

In [9]:
# X_train.columns.values

In [61]:
X_train.drop(['incident_id', 'row_id', 'vehicle_id', 'occupant_id'], axis = 1, inplace = True)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
# X_train.columns.values

In [76]:
#X_valid.reset_index(inplace = True)

In [66]:
X_valid.drop(['incident_id', 'row_id', 'vehicle_id', 'occupant_id'], axis = 1, inplace = True)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [67]:
print(len(X_train.columns.values))
print(len(X_valid.columns.values))

60
61


In [74]:
set(X_valid.columns.values).difference(set(X_train.columns.values))

{'index'}

In [77]:
X_valid.drop(['index'], axis = 1, inplace = True)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [78]:
len(X_valid.columns.values)

60

In [11]:
# X_valid.columns.values

In [80]:
y_train.head()

injury
0       4
1       0
2       0
3       3
4       4

In [81]:
# fit Lightgbm model

import lightgbm as lgb

lgbtrain = lgb.Dataset(X_train, label = y_train)
lgbvalid = lgb.Dataset(X_valid, label = y_valid)

In [82]:

params = {'objective': 'multiclass',
          'num_class': 5,
         'metric': 'multi_logloss',
         'num_leaves': 31,
          'max_depth': 7,
         'learning_rate': 0.05,
         'feature_fraction': 0.9,
         'bagging_fraction': 0.8,
         'bagging_freq': 5,
         'verbose': 1}

In [86]:

%%time

mod_lgb_1 = lgb.train(params = params,
                   train_set = lgbtrain,
                   num_boost_round = 5000,
                   valid_sets = lgbvalid,
                   early_stopping_rounds = 50,
                   verbose_eval = 50)



Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 0.992096
[100]	valid_0's multi_logloss: 0.930797
[150]	valid_0's multi_logloss: 0.909289
[200]	valid_0's multi_logloss: 0.897272
[250]	valid_0's multi_logloss: 0.89013
[300]	valid_0's multi_logloss: 0.885265
[350]	valid_0's multi_logloss: 0.882178
[400]	valid_0's multi_logloss: 0.87979
[450]	valid_0's multi_logloss: 0.878336
[500]	valid_0's multi_logloss: 0.87644
[550]	valid_0's multi_logloss: 0.875607
[600]	valid_0's multi_logloss: 0.874579
[650]	valid_0's multi_logloss: 0.87434
[700]	valid_0's multi_logloss: 0.874415
Early stopping, best iteration is:
[657]	valid_0's multi_logloss: 0.874216
Wall time: 2min 56s


In [85]:
X_train.columns

Index(['total_veh_in_inc', 'total_occ_in_inc', 'total_occ_in_veh',
       'actual_veh_in_inc', 'actual_per_in_inc', 'actual_occ_in_veh',
       'actual_ped', 'inc_day', 'inc_region', 'inc_daylight', 'inc_weather',
       'veh_roadcond', 'veh_roadslope', 'veh_lanes', 'veh_lanediv',
       'veh_speedlim', 'occ_age', 'occ_sex', 'occ_role', 'occ_position',
       'occ_seatbelt', 'occ_airbag', 'occ_alcohol', 'veh_owner', 'veh_type',
       'veh_style', 'veh_weight', 'veh_manu', 'veh_doors', 'veh_wheels',
       'veh_drive', 'veh_fuel', 'veh_eng1', 'veh_eng2', 'veh_trailer',
       'drv_height', 'drv_weight', 'drv_licence', 'drv_status', 'drv_crash',
       'drv_pts', 'drv_dui', 'drv_spd', 'drv_alcohol', 'inc_type',
       'inc_speeding', 'veh_speed', 'veh_braking', 'veh_stability',
       'veh_movement', 'veh_impact', 'inc_descrip', 'veh_descrip',
       'veh_damage', 'veh_age_new', 'drv_ban1_new', 'drv_ban2_new',
       'inc_month_new', 'inc_hour_new', 'veh_inc_maxage'],
      dtype='objec

In [102]:
# lightgbm with categorical features

feature_name = X_train.columns.values
categorical_feature = ['inc_day', 'inc_region', 'inc_daylight', 'inc_weather', 'veh_roadcond', 'veh_roadslope', 'veh_lanediv',
                      'occ_age', 'occ_sex', 'occ_role', 'occ_position', 'occ_seatbelt', 'occ_airbag', 'veh_owner',
                      'veh_type', 'veh_style', 'veh_weight', 'veh_manu', 'veh_drive', 'veh_fuel', 'veh_trailer', 'drv_licence',
                       'drv_status', 'inc_type', 'veh_braking', 'veh_stability', 'veh_movement', 'veh_impact', 'inc_descrip',
                       'veh_descrip', 'veh_damage', 'inc_month_new']


lgbtrain = lgb.Dataset(X_train, label = y_train, feature_name = list(feature_name), categorical_feature = categorical_feature)
lgbvalid = lgb.Dataset(X_valid, label = y_valid, feature_name = list(feature_name), categorical_feature = categorical_feature)


params_2 = {'objective': 'multiclass',
           'num_class': 5,
           'metric': 'multi_logloss',
           'num_leaves': 31,
           'max_depth': 7,
           'learning_rate': 0.05,
           'feature_fraction': 0.9,
           'bagging_fraction': 0.8,
           'bagging_freq': 5,
           'verbose': 1}

In [12]:
# feature_name

In [13]:
# list(feature_name)

In [14]:
# set(feature_name).difference(set(categorical_feature))

In [95]:
len(categorical_feature)

32

In [97]:
len(feature_name)

60

In [103]:
mod_lgb_2 = lgb.train(params = params_2,
                     train_set = lgbtrain,
                     num_boost_round = 5000,
                     valid_sets = lgbvalid,
                     early_stopping_rounds = 50,
                     verbose_eval = 40)

C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\LUY1\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[40]	valid_0's multi_logloss: 1.00669
[80]	valid_0's multi_logloss: 0.930659
[120]	valid_0's multi_logloss: 0.90851
[160]	valid_0's multi_logloss: 0.898025
[200]	valid_0's multi_logloss: 0.892605
[240]	valid_0's multi_logloss: 0.888655
[280]	valid_0's multi_logloss: 0.8862
[320]	valid_0's multi_logloss: 0.884984
[360]	valid_0's multi_logloss: 0.884018
[400]	valid_0's multi_logloss: 0.883016
[440]	valid_0's multi_logloss: 0.882738
[480]	valid_0's multi_logloss: 0.882884
Early stopping, best iteration is:
[430]	valid_0's multi_logloss: 0.882612
